In [1]:
import subprocess
import threading
import time
import cv2
import ipywidgets as widgets
from IPython.display import display

PAUSE = 0
PLAY_FORWARD = 1
PLAY_BACKWARD = -1
video_path = 'video.mp4'

def get_i_frames():
    cmd = ['ffprobe', '-select_streams', 'v', '-show_frames', '-show_entries',\
           'frame=pict_type', '-of', 'csv', video_path]
    ret = subprocess.run(cmd, stdout = subprocess.PIPE, stderr = subprocess.PIPE, text = True)
    lines = ret.stdout.splitlines()
    i_frames = []
    
    for i, line in enumerate(lines):
        if line.split(',')[1] == 'I':
            i_frames.append(i)
    return i_frames

# ------------------------------
# UI callbacks
def set_progress(change):
    global current_frame, ignore_slider_change
    
    if change['name'] == 'value':
        if ignore_slider_change:
            ignore_slider_change = False
            return
        current_frame = change['new']

def set_pause(b):
    global play_state
    
    play_state = PAUSE

def set_forward(b):
    global play_state
    
    play_state = PLAY_FORWARD

def set_backward(b):
    global play_state
    
    play_state = PLAY_BACKWARD
# ------------------------------

In [2]:
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_delay = 1 / cap.get(cv2.CAP_PROP_FPS)
i_frames = get_i_frames()
current_frame = 0
play_state = PAUSE
ignore_slider_change = False #slider is changed by user

# ------------------------------
# UIs
video_widget = widgets.Image(format = 'jpeg')
slider = widgets.IntSlider(value = 0, min = 0, max = total_frames - 1, description = 'Frame')
btn_pause = widgets.Button(description = '⏸')
btn_forward = widgets.Button(description = '▶')
btn_backward = widgets.Button(description = '◀')

slider.observe(set_progress)
btn_pause.on_click(set_pause)
btn_forward.on_click(set_forward)
btn_backward.on_click(set_backward)
# ------------------------------

def display_frame(frame):
    _, img = cv2.imencode('.jpg', frame)
    
    video_widget.value = img.tobytes()

def find_previous_i_frame(index):
    for i in reversed(i_frames):
        if i <= index:
            return i
    return 0

def play_video():
    global current_frame, play_state, ignore_slider_change
    
    while True:
        if play_state == PAUSE:
            time.sleep(.1)
        elif play_state == PLAY_FORWARD:
            if current_frame >= total_frames - 1:
                current_frame = total_frames - 1
                play_state = PAUSE
                continue
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
            
            ret, frame = cap.read()
            
            if not ret:
                break
            display_frame(frame)
            slider.value = current_frame + 1
        elif play_state == PLAY_BACKWARD:
            if current_frame <= 0:
                current_frame = 0
                play_state = PAUSE
                continue
            
            target = current_frame
            start_frame = find_previous_i_frame(target - 1)
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            
            frames = []
            for _ in range(current_frame - start_frame):
                ret, frame = cap.read()
                
                if not ret:
                    break
                frames.append(frame)
            for i in reversed(frames):
                if play_state != PLAY_BACKWARD or target != current_frame:
                    break
                display_frame(i)
                current_frame = target - 1
                target = current_frame
                ignore_slider_change = True
                slider.value = current_frame
                time.sleep(frame_delay)

In [3]:
i_frames

[0, 91, 182, 273, 364, 455, 545, 636]

In [4]:
thread = threading.Thread(target = play_video, daemon = True)

display(widgets.VBox([video_widget, widgets.HBox([btn_backward, btn_pause, btn_forward]), slider]))
thread.start()